Регулярки

In [1]:
from re import compile, sub

def cut(data):
    r = compile(r'<.*?>|>>\d*|\(OP\)|&#(\d*);|&quot;|&gt;|(http|https):.*')
    return r.sub('', punctuate_word(punctuate_sent((data))))

def punctuate_sent(data):
    r = compile(r'([a-zA-Zа-яА-Я])([.!\?])')
    return r.sub(r'\1. ', data)

def punctuate_word(data):
    r = compile(r'([a-zA-Zа-яА-Я])([,])')
    return r.sub(r'\1, ', data)

def pos_cut(s):
    s = sub(r"ADJF", "ADJ", s)
    s = sub(r"ADVB", "ADV", s)
    s = sub(r"INFN", "VERB", s)
    return s

def topic_cut(word):
    return sub('[^a-zA-Zа-яА-Я]', '', word)

def check_for_rus_lan(word):
    regex = compile('[А-Яа-я]+$')
    if regex.match(word):
        return True
    return False

def check_dirty(word):
    regexp = compile(r'уеб|хуё|ёб|пидо|иби|хуй|еба|бля|еби|хуе|хуи|пиз|срал|velf|оеб')
    if regexp.search(word):
          return True
    return False

Получение списка тредов доски

In [3]:
from requests import get, exceptions

DVACH = 'https://2ch.hk/'
BOARD = 'pr'

dvach_page = get(DVACH + BOARD + '/catalog.json').json()
threads = [i['num'] for i in dvach_page['threads']]

Формирование вектора длин постов

In [ ]:
posts_lengths = []

for thread in threads:
    try:
        thread = get(DVACH +  BOARD + '/res/' + thread + '.json',  timeout=5).json()
        [posts_lengths.append(len(list(tokenize(i['comment'])))) for i in thread['threads'][0]['posts'] if len(i['comment']) > 2]
    except:
        pass

Сериализация

In [8]:
subdir = 'pickle'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, BOARD + '_posts_lengths.pickle')

with open(file_path, 'wb') as f:
    pickle.dump(posts_lengths, f)

Формирование корпуса сообщений доски

In [5]:
from itertools import chain
from nltk.tokenize import sent_tokenize

documents = []

for every_thread in threads:
    #try:
    thread = get(DVACH +  BOARD + '/res/' + every_thread + '.json', timeout=5).json()
    [documents.append(sent_tokenize(cut(i['comment']))) for i in thread['threads'][0]['posts'] if len(cut(i['comment'])) > 2]
    #except:
    #    pass
documents = list(chain.from_iterable(documents))

In [6]:
len(documents)

127642

Сериализация и мердж корпуса

In [7]:
from os import path, makedirs
from pickle import dump

subdir = 'pickle'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, BOARD + '.pickle')

with open(file_path, 'wb') as f:
    dump(documents, f)